### Installing dependencies...

In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install downcast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importing libraries

In [4]:
import opendatasets as od
import pandas as pd

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from downcast import reduce

### Downloading dataset from kaggle

In [6]:
# {"username":"shivangshrivastav","key":"ef4ce21fcc0c84d6f64ba43bb4c67d1c"}

In [7]:

link = 'https://www.kaggle.com/competitions/competitive-data-science-predict-future-sales/data'
od.download(link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shivangshrivastav
Your Kaggle Key: ··········


100%|██████████| 15.1M/15.1M [00:00<00:00, 176MB/s]


Extracting archive ./competitive-data-science-predict-future-sales/competitive-data-science-predict-future-sales.zip to ./competitive-data-science-predict-future-sales


### Loading Data

In [8]:
train_data       = pd.read_csv('/content/competitive-data-science-predict-future-sales/sales_train.csv');
test_data        = pd.read_csv('/content/competitive-data-science-predict-future-sales/test.csv');
submission       = pd.read_csv('/content/competitive-data-science-predict-future-sales/sample_submission.csv');
items            = pd.read_csv('/content/competitive-data-science-predict-future-sales/items.csv');
item_categories  = pd.read_csv('/content/competitive-data-science-predict-future-sales/item_categories.csv');
shops            = pd.read_csv('/content/competitive-data-science-predict-future-sales/shops.csv');

### Using only those shops in train data, which are there in test data

In [9]:
# Only use shops in train data that are in test data

unique_test_shops = test_data["shop_id"].unique()
train_data = train_data[train_data["shop_id"].isin(unique_test_shops)]

print(f"Number of Unique Shops in Test Data:{len(unique_test_shops)}\nNumber of Unique Shops in Sales Data:{len(train_data['shop_id'].unique())}")

Number of Unique Shops in Test Data:42
Number of Unique Shops in Sales Data:42


###Reducing Size of Data Frame

In [10]:
def reduce_size(df):

  x=df.memory_usage(deep=True).sum()/1024**2 
  df=reduce(df)
  y=df.memory_usage(deep=True).sum()/1024**2
  print("Memory usage decreased to {:.2f} Mb ({:.1f} % reduction)".format(y,100*(x-y)/x))

  return df

### Exploring Loaded Datase

In [11]:
submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [12]:
print(train_data.shape)
train_data.head()

(2413246, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [13]:
print(test_data.shape)
test_data.head()

(214200, 3)


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [14]:
print(item_categories.shape)
item_categories.head()

(84, 2)


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [15]:
print(shops.shape)
shops.head()

(60, 2)


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [16]:
print(shops.shop_name.nunique())
print(shops.shop_id.nunique())

60
60


In [17]:
print(items.shape)
items.head()

(22170, 3)


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [18]:
items.isna().sum()

item_name           0
item_id             0
item_category_id    0
dtype: int64

In [19]:
print(items.item_name.nunique())
print(items.item_id.nunique())
print(items.item_category_id.nunique())

22170
22170
84


In [20]:
train_data['Date'] = pd.to_datetime(train_data['date'])
train_data.drop('date',axis=1,inplace = True)
train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Date
0,0,59,22154,999.00,1.0,2013-02-01
1,0,25,2552,899.00,1.0,2013-03-01
2,0,25,2552,899.00,-1.0,2013-05-01
3,0,25,2554,1709.05,1.0,2013-06-01
4,0,25,2555,1099.00,1.0,2013-01-15


In [21]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2413246 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date_block_num  int64         
 1   shop_id         int64         
 2   item_id         int64         
 3   item_price      float64       
 4   item_cnt_day    float64       
 5   Date            datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 128.9 MB


In [22]:
train_data.shape

(2413246, 6)

### Extracting City Name and Encoding it

In [23]:
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import StandardScaler

In [24]:
# Shops

def shop_name2city(sn):
    sn = sn.split()[0]
    if sn == 'Цифровой' or sn == 'Интернет-магазин': sn = 'Internet'
    if sn[0] == '!': sn = sn[1:]  
    return sn

shops['city'] = shops['shop_name'].apply(shop_name2city)
shops['city_enc'] = LabelEncoder().fit_transform(shops['city']).astype('int8')
shops.head()

,shop_name,shop_id,city,city_enc
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,28
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,28
2,"Адыгея ТЦ ""Мега""",2,Адыгея,1
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,2
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,3


### Extracting Category names

In [25]:
item_categories['Category'] = item_categories['item_category_name'].str.split(n=1, expand=True)[0]
item_categories['Category_id'] = LabelEncoder().fit_transform(item_categories['Category'])
item_categories.head()

,item_category_name,item_category_id,Category,Category_id
0,PC - Гарнитуры/Наушники,0,PC,0
1,Аксессуары - PS2,1,Аксессуары,1
2,Аксессуары - PS3,2,Аксессуары,1
3,Аксессуары - PS4,3,Аксессуары,1
4,Аксессуары - PSP,4,Аксессуары,1


In [26]:
item_categories.Category_id.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [27]:
item_categories.Category.unique()

array(['PC', 'Аксессуары', 'Билеты', 'Доставка', 'Игровые', 'Игры',
       'Карты', 'Кино', 'Книги', 'Музыка', 'Подарки', 'Программы',
       'Служебные', 'Чистые', 'Элементы'], dtype=object)

In [28]:
all_about_items = pd.merge(items, item_categories,on='item_category_id',how='left')
all_about_items.head()

,item_name,item_id,item_category_id,item_category_name,Category,Category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино,7
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра),Программы,11
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,7
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,7
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD,Кино,7


In [29]:
item_categories.Category.value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: Category, dtype: int64

In [30]:
train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Date
0,0,59,22154,999.00,1.0,2013-02-01
1,0,25,2552,899.00,1.0,2013-03-01
2,0,25,2552,899.00,-1.0,2013-05-01
3,0,25,2554,1709.05,1.0,2013-06-01
4,0,25,2555,1099.00,1.0,2013-01-15


In [31]:
print(train_data.shape)
train_data.head()

(2413246, 6)


,date_block_num,shop_id,item_id,item_price,item_cnt_day,Date
0,0,59,22154,999.00,1.0,2013-02-01
1,0,25,2552,899.00,1.0,2013-03-01
2,0,25,2552,899.00,-1.0,2013-05-01
3,0,25,2554,1709.05,1.0,2013-06-01
4,0,25,2555,1099.00,1.0,2013-01-15


In [32]:
train_data['month'] = train_data['Date'].dt.month.astype(np.int8)
train_data.drop('Date',axis=1,inplace=True)
train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,0,59,22154,999.00,1.0,2
1,0,25,2552,899.00,1.0,3
2,0,25,2552,899.00,-1.0,5
3,0,25,2554,1709.05,1.0,6
4,0,25,2555,1099.00,1.0,1


In [33]:
train_data.shape

(2413246, 6)

### Removing negative values from item_cnt after grouping them

In [34]:
index_feats = ["date_block_num", "shop_id", "item_id"]

In [35]:
item_total_cnt = train_data.groupby(['shop_id', 'item_id'])['item_cnt_day'].sum()
items_to_drop = item_total_cnt[item_total_cnt <= 0].index
train_data = train_data[~train_data.set_index(['shop_id', 'item_id']).index.isin(items_to_drop)]
            
item_block_cnt = train_data.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum()
items_to_drop = item_block_cnt[item_block_cnt <= 0].index
train_data = train_data[~train_data.set_index(['date_block_num', 'shop_id', 'item_id']).index.isin(items_to_drop)]

### Removing the outliers in 'item_price' and 'item_cnt_day'

In [36]:
train_data = train_data[(train_data['item_price'] > 0) &  (train_data['item_price'] < 51000)]
train_data = train_data[train_data['item_cnt_day'] <= 1000]

In [37]:
train_data.shape

(2407956, 6)

### Using itertools to create new dataframe with equal number of shops for every block

In [38]:
from itertools import product

train = []

for i in range(0,34):
    
    cur_shops = train_data.loc[train_data["date_block_num"] == i, "shop_id"].unique()
    
    cur_items = train_data.loc[train_data["date_block_num"] == i, "item_id"].unique()
    
    train.append(np.array(list(product(*[[i],cur_shops, cur_items]))))
    
index_feats = ["date_block_num", "shop_id", "item_id"]
train = pd.DataFrame(np.vstack(train), columns=index_feats)
train = reduce_size(train)
train

Memory usage decreased to 32.76 Mb (83.3 % reduction)


,date_block_num,shop_id,item_id
0,0,59,22154
1,0,59,2554
2,0,59,2555
3,0,59,2564
4,0,59,2565
...,...,...,...
8587185,33,21,7635
8587186,33,21,7638
8587187,33,21,7640
8587188,33,21,7632


### Checking total items Sold per month

In [39]:
idx_columns = ['date_block_num', 'shop_id', 'item_id']
cmin = 0
cmax = 20

In [40]:
temp = train_data.groupby(idx_columns).agg({'item_price'  : np.mean, 
                                                               'item_cnt_day': np.sum})
temp.reset_index(inplace=True)
temp = temp.rename(columns={'item_cnt_day': 'item_cnt_month'})
        
train_df = pd.merge(train, temp, on=idx_columns, how='left')
        
train_df['item_cnt_month'] = train_df['item_cnt_month'].astype('float32').fillna(0.).clip(cmin, cmax)
train_df['item_price'] = train_df['item_price'].astype('float32').fillna(0.)

train_df

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,59,22154,999.0,1.0
1,0,59,2554,0.0,0.0
2,0,59,2555,0.0,0.0
3,0,59,2564,0.0,0.0
4,0,59,2565,0.0,0.0
...,...,...,...,...,...
8587185,33,21,7635,0.0,0.0
8587186,33,21,7638,0.0,0.0
8587187,33,21,7640,0.0,0.0
8587188,33,21,7632,0.0,0.0


In [41]:
train_data

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,0,59,22154,999.00,1.0,2
3,0,25,2554,1709.05,1.0,6
4,0,25,2555,1099.00,1.0,1
5,0,25,2564,349.00,1.0,10
6,0,25,2565,549.00,1.0,2
...,...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0,10
2935845,33,25,7460,299.00,1.0,9
2935846,33,25,7459,349.00,1.0,10
2935847,33,25,7440,299.00,1.0,10


In [42]:
train_df = train_df.fillna(0)

### Creating Count of Items sold per month ==> know as 'item_rate_month'

In [43]:
# Add column for count of items sold.

df_tmp = train_data[train_data['item_cnt_day'] > 0].groupby(idx_columns).agg({'item_cnt_day': 'count'})
df_tmp.reset_index(inplace=True)
df_tmp = df_tmp.rename(columns={'item_cnt_day': 'item_rate_month'})
        
train_df = pd.merge(train_df, df_tmp, on=idx_columns, how='left')
        
train_df['item_rate_month'] = train_df['item_rate_month'].astype('float32').fillna(0.)
   

train_df = reduce_size(train_df)
train_df.sample(5)

Memory usage decreased to 147.41 Mb (25.0 % reduction)


,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month
120646,0,10,5491,0.0,0.0,0.0
8024912,31,55,1215,0.0,0.0,0.0
3875948,14,50,505,0.0,0.0,0.0
5845048,21,42,8986,0.0,0.0,0.0
2816963,10,41,8164,0.0,0.0,0.0


In [44]:
train_df.item_cnt_month.nunique()

21

In [45]:
train_df.item_rate_month.value_counts()

0.0     7266441
1.0      885703
2.0      223430
3.0       86517
4.0       43806
5.0       25507
6.0       16133
7.0       10578
8.0        7277
9.0        5103
10.0       3677
11.0       2761
12.0       2123
13.0       1627
14.0       1255
15.0        937
16.0        781
17.0        620
18.0        492
19.0        381
20.0        319
21.0        231
23.0        190
22.0        186
30.0        171
24.0        153
29.0        144
27.0        140
26.0        132
31.0        131
25.0        127
28.0        117
Name: item_rate_month, dtype: int64

In [46]:
import gc

gc.collect()

18

## Creating Average Count features.....

### Average items sold per month ==> sum of items per month / total item count per month

In [47]:
group = train_df.groupby( ["date_block_num"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_avg_item_cnt"]
group.reset_index(inplace = True)


In [48]:
train_df = pd.merge(train_df, group, on = ["date_block_num"], how = "left")
train_df.date_avg_item_cnt = train_df["date_avg_item_cnt"].astype(np.float16)
train_df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt
0,0,59,22154,999.0,1.0,1.0,0.359863
1,0,59,2554,0.0,0.0,0.0,0.359863
2,0,59,2555,0.0,0.0,0.0,0.359863
3,0,59,2564,0.0,0.0,0.0,0.359863
4,0,59,2565,0.0,0.0,0.0,0.359863


### Average item count per month

In [49]:
group = train_df.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)


In [50]:

train_df = pd.merge(train_df, group, on=['date_block_num','item_id'], how='left')
train_df.date_item_avg_item_cnt = train_df['date_item_avg_item_cnt'].astype(np.float16)

### Average item count per month for every shop

In [51]:
group = train_df.groupby( ["date_block_num","shop_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_avg_item_cnt"]
group.reset_index(inplace = True)

train_df = pd.merge(train_df, group, on = ["date_block_num","shop_id"], how = "left")
train_df.date_avg_item_cnt = train_df["date_shop_avg_item_cnt"].astype(np.float16)

### Average item count per month for every item_id and every shop

In [52]:
group = train_df.groupby( ["date_block_num","shop_id","item_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_item_avg_item_cnt"]
group.reset_index(inplace = True)

train_df = pd.merge(train_df, group, on = ["date_block_num","shop_id","item_id"], how = "left")
train_df.date_avg_item_cnt = train_df["date_shop_item_avg_item_cnt"].astype(np.float16)

In [53]:
train_df

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt
0,0,59,22154,999.0,1.0,1.0,1.0,0.484863,0.267578,1.0
1,0,59,2554,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0
2,0,59,2555,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0
3,0,59,2564,0.0,0.0,0.0,0.0,0.121216,0.267578,0.0
4,0,59,2565,0.0,0.0,0.0,0.0,0.060608,0.267578,0.0
...,...,...,...,...,...,...,...,...,...,...
8587185,33,21,7635,0.0,0.0,0.0,0.0,0.023804,0.330811,0.0
8587186,33,21,7638,0.0,0.0,0.0,0.0,0.023804,0.330811,0.0
8587187,33,21,7640,0.0,0.0,0.0,0.0,0.023804,0.330811,0.0
8587188,33,21,7632,0.0,0.0,0.0,0.0,0.023804,0.330811,0.0


### Reducing size of data frames

In [54]:
train_df = reduce_size(train_df)


Memory usage decreased to 212.92 Mb (0.0 % reduction)


### Merging Data Frames

In [55]:
train_df = pd.merge(train_df, shops, on="shop_id", how="left")
train_df = pd.merge(train_df, all_about_items, on="item_id", how="left")
train_df.drop(['shop_name','city','item_name','item_category_name','Category'],axis=1,inplace=True)
gc.collect()
train_df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,city_enc,item_category_id,Category_id
0,0,59,22154,999.0,1.0,1.0,1.0,0.484863,0.267578,1.0,29,37,7
1,0,59,2554,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,29,58,9
2,0,59,2555,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,29,56,9
3,0,59,2564,0.0,0.0,0.0,0.0,0.121216,0.267578,0.0,29,59,9
4,0,59,2565,0.0,0.0,0.0,0.0,0.060608,0.267578,0.0,29,56,9


## Creating test data frame and merging with train Data frame

### Creating Test Data Frame

In [56]:
test_data['date_block_num'] = 34
test_data['item_cnt_month'] = 0
test_data['item_price'] = 0
test_data['date_avg_item_cnt'] = 0
test_data['date_item_avg_item_cnt'] = 0
test_data['date_shop_avg_item_cnt'] = 0

test_data['date_shop_item_avg_item_cnt'] = 0
test_data['date_city_avg_item_cnt'] = 0
test_data['date_item_city_avg_item_cnt'] = 0
test_data['item_avg_item_price'] = 0
test_data['date_item_avg_item_price'] = 0


test_data['item_rate_month'] = 0
test_data.drop('ID',axis=1,inplace=True)
test_data.head()


,shop_id,item_id,date_block_num,item_cnt_month,item_price,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,date_city_avg_item_cnt,date_item_city_avg_item_cnt,item_avg_item_price,date_item_avg_item_price,item_rate_month
0,5,5037,34,0,0,0,0,0,0,0,0,0,0,0
1,5,5320,34,0,0,0,0,0,0,0,0,0,0,0
2,5,5233,34,0,0,0,0,0,0,0,0,0,0,0
3,5,5232,34,0,0,0,0,0,0,0,0,0,0,0
4,5,5268,34,0,0,0,0,0,0,0,0,0,0,0


### Merging test data frame with respective shops and items dataframe

In [57]:
test_data = pd.merge(test_data, shops, on="shop_id", how="left")
test_data = pd.merge(test_data, all_about_items, on="item_id", how="left")
test_data.drop(['shop_name','city','item_name','item_category_name','Category'],axis=1,inplace=True)

In [58]:
test_data.shape

(214200, 17)

In [59]:
train_df.shape

(8587190, 13)

In [60]:
test_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'item_price',
       'date_avg_item_cnt', 'date_item_avg_item_cnt', 'date_shop_avg_item_cnt',
       'date_shop_item_avg_item_cnt', 'date_city_avg_item_cnt',
       'date_item_city_avg_item_cnt', 'item_avg_item_price',
       'date_item_avg_item_price', 'item_rate_month', 'city_enc',
       'item_category_id', 'Category_id'],
      dtype='object')

In [61]:
train_df.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_month',
       'item_rate_month', 'date_avg_item_cnt', 'date_item_avg_item_cnt',
       'date_shop_avg_item_cnt', 'date_shop_item_avg_item_cnt', 'city_enc',
       'item_category_id', 'Category_id'],
      dtype='object')

### Before merging train and test dataframe, check whether features of both the columns are same or not

In [62]:
for x in train_df.columns:
  if x not in test_data.columns:
    print(x)

### Re-arranging the test data frame features according to train data frame features

In [63]:
test_data = test_data[['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_month',
       'item_rate_month', 'date_avg_item_cnt', 'date_item_avg_item_cnt',
       'date_shop_avg_item_cnt', 'date_shop_item_avg_item_cnt', 'city_enc',
       'item_category_id', 'Category_id', 'date_city_avg_item_cnt',
       'date_item_city_avg_item_cnt', 'item_avg_item_price',
       'date_item_avg_item_price']]

In [64]:
test_data.shape

(214200, 17)

### Merging Test and Train data frame to form the final data frame 'df'

In [65]:
df = train_df.append(test_data)
df.reset_index(drop=True,inplace=True)
df

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,city_enc,item_category_id,Category_id,date_city_avg_item_cnt,date_item_city_avg_item_cnt,item_avg_item_price,date_item_avg_item_price
0,0,59,22154,999.0,1.0,1.0,1.0,0.484863,0.267578,1.0,29,37,7,NaN,NaN,NaN,NaN
1,0,59,2554,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,29,58,9,NaN,NaN,NaN,NaN
2,0,59,2555,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,29,56,9,NaN,NaN,NaN,NaN
3,0,59,2564,0.0,0.0,0.0,0.0,0.121216,0.267578,0.0,29,59,9,NaN,NaN,NaN,NaN
4,0,59,2565,0.0,0.0,0.0,0.0,0.060608,0.267578,0.0,29,56,9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801385,34,45,18454,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,55,9,0.0,0.0,0.0,0.0
8801386,34,45,16188,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,64,10,0.0,0.0,0.0,0.0
8801387,34,45,15757,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,55,9,0.0,0.0,0.0,0.0
8801388,34,45,19648,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,40,7,0.0,0.0,0.0,0.0


In [66]:
df[df['date_block_num'] == 34]

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,city_enc,item_category_id,Category_id,date_city_avg_item_cnt,date_item_city_avg_item_cnt,item_avg_item_price,date_item_avg_item_price
8587190,34,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,19,5,0.0,0.0,0.0,0.0
8587191,34,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,55,9,0.0,0.0,0.0,0.0
8587192,34,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,19,5,0.0,0.0,0.0,0.0
8587193,34,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,23,5,0.0,0.0,0.0,0.0
8587194,34,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,20,5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801385,34,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,55,9,0.0,0.0,0.0,0.0
8801386,34,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,64,10,0.0,0.0,0.0,0.0
8801387,34,45,15757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,55,9,0.0,0.0,0.0,0.0
8801388,34,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,40,7,0.0,0.0,0.0,0.0


In [67]:
df[df['date_block_num'] == 34].shape

(214200, 17)

## Creating Additional Features...

### Creating additional feature of 'First_Sale_Date'

In [68]:
df['first_sale_date'] = df.groupby('item_id')['date_block_num'].min()
        
df['first_sale_date'].fillna(34, inplace=True)
df['first_sale_date'] = df['first_sale_date'].astype('int8')

In [69]:
df[df['date_block_num'] == 34].shape

(214200, 18)

### Creating new feature named 'month'

In [70]:
df['month'] = (df['date_block_num']%12 + 1).astype('int8')

In [71]:
df[df['date_block_num'] == 34].shape

(214200, 19)

In [72]:
df

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,city_enc,item_category_id,Category_id,date_city_avg_item_cnt,date_item_city_avg_item_cnt,item_avg_item_price,date_item_avg_item_price,first_sale_date,month
0,0,59,22154,999.0,1.0,1.0,1.0,0.484863,0.267578,1.0,29,37,7,NaN,NaN,NaN,NaN,34,1
1,0,59,2554,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,29,58,9,NaN,NaN,NaN,NaN,15,1
2,0,59,2555,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,29,56,9,NaN,NaN,NaN,NaN,34,1
3,0,59,2564,0.0,0.0,0.0,0.0,0.121216,0.267578,0.0,29,59,9,NaN,NaN,NaN,NaN,34,1
4,0,59,2565,0.0,0.0,0.0,0.0,0.060608,0.267578,0.0,29,56,9,NaN,NaN,NaN,NaN,34,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801385,34,45,18454,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,55,9,0.0,0.0,0.0,0.0,34,11
8801386,34,45,16188,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,64,10,0.0,0.0,0.0,0.0,34,11
8801387,34,45,15757,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,55,9,0.0,0.0,0.0,0.0,34,11
8801388,34,45,19648,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,20,40,7,0.0,0.0,0.0,0.0,34,11


In [73]:
df[df['date_block_num'] == 34].shape

(214200, 19)

### Creating first instance of shop appearance in the months

In [74]:
# Shop start block number
first_instance_of_shop = df.groupby('shop_id')['date_block_num'].min().astype('int8').reset_index()
first_instance_of_shop.rename(columns={"date_block_num": "first_instance_of_shop"}, inplace=True);
df = df.merge(first_instance_of_shop, how = 'left', on = 'shop_id')


### Creating new feature named 'new_item'

In [75]:
df['new_item'] = df['first_sale_date'] ==  df['date_block_num']

In [76]:
df['first_sale_date'] = abs(df['date_block_num'] - df['first_sale_date'])
df['first_instance_of_shop'] = abs(df['date_block_num'] - df['first_instance_of_shop'])
gc.collect()

0

In [77]:
df = reduce_size(df)

Memory usage decreased to 344.14 Mb (73.5 % reduction)


In [78]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,...,item_category_id,Category_id,date_city_avg_item_cnt,date_item_city_avg_item_cnt,item_avg_item_price,date_item_avg_item_price,first_sale_date,month,first_instance_of_shop,new_item
0,0,59,22154,999.0,1.0,1.0,1.0,0.484863,0.267578,1.0,...,37,7,NaN,NaN,NaN,NaN,34,1,0,False
1,0,59,2554,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,...,58,9,NaN,NaN,NaN,NaN,15,1,0,False
2,0,59,2555,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,...,56,9,NaN,NaN,NaN,NaN,34,1,0,False
3,0,59,2564,0.0,0.0,0.0,0.0,0.121216,0.267578,0.0,...,59,9,NaN,NaN,NaN,NaN,34,1,0,False
4,0,59,2565,0.0,0.0,0.0,0.0,0.060608,0.267578,0.0,...,56,9,NaN,NaN,NaN,NaN,34,1,0,False


## Adding mean Features...

### Creating function for adding item means to data frame

In [79]:
def add_item_means(df, feature = None):
    item_mean_features = []
    print('Adding item means.......')
            
    if feature == None :
        group_items = ['date_block_num','item_id']
        feature = 'item_cnt'
    else:
        group_items = ['date_block_num','item_id'] + [feature]
                
    feature_mean_name = feature + '_mean'
                
    print('Adding item means for', feature, '...')
            
    df_tmp = df.groupby(group_items)['item_cnt_month'].mean()
    df_tmp = df_tmp.reset_index().rename(columns = {'item_cnt_month': feature_mean_name})
            
    df = pd.merge(df, df_tmp, on=group_items, how='left')
            
    item_mean_features.append(feature_mean_name)
            
    del df_tmp
    gc.collect()        
    return df, item_mean_features

### Creating function to adding shop means to data frame

In [80]:
def add_shop_means(df, feature):
    shop_mean_features = []
    print('Adding shop means.......')
            
    group_items = ['date_block_num', 'shop_id'] + [feature]
                
    feature_mean_name = feature + '_mean'
                
    print('Adding shop means for', feature, '...')
            
    df_tmp = df.groupby(group_items)['item_cnt_month'].mean()
    df_tmp = df_tmp.reset_index().rename(columns = {'item_cnt_month': feature_mean_name})
            
    df = pd.merge(df, df_tmp, on=group_items, how='left')
            
    shop_mean_features.append(feature)
            
    del df_tmp
    gc.collect()        
    return df, shop_mean_features

### Adding item and shop means.....................

In [81]:
%%time

df, item_mean_features = add_item_means(df)
df, item_mean_features = add_item_means(df,'item_category_id')
df, shop_mean_features = add_shop_means(df,'city_enc')

Adding item means.......
Adding item means for item_cnt ...
Adding item means.......
Adding item means for item_category_id ...
Adding shop means.......
Adding shop means for city_enc ...
CPU times: user 10.8 s, sys: 120 ms, total: 11 s
Wall time: 10.9 s


In [82]:
print(item_mean_features)

['item_category_id_mean']


In [83]:
df = reduce_size(df)

Memory usage decreased to 394.50 Mb (0.0 % reduction)


## Adding Lag Features...

### Creating Function for item lag features

In [84]:
def add_item_lags(df, feature_name,nlags , lag_names_to_clip=[]):
        
        print('Adding item lags for', feature_name, '...')
        
    
        df_tmp = df[['date_block_num', 'shop_id', 'item_id', feature_name]]
    
        for i in range(nlags, 0, -1):
            lag_feature_name = feature_name +'_lag-' + str(i)
            lag_names_to_clip.append(lag_feature_name)
            df_shifted = df_tmp.copy()
            df_shifted.columns = ['date_block_num', 'shop_id', 'item_id', lag_feature_name]
            df_shifted['date_block_num'] += i
            df = pd.merge(df, df_shifted.drop_duplicates(), on=['date_block_num', 'shop_id', 'item_id'], how='left')
            
            df[lag_feature_name] = df[lag_feature_name].fillna(0).astype('float32')
        
        del df_tmp
        del df_shifted
        return df, lag_names_to_clip

### Creating function for shop lag features

In [85]:
def add_shop_lags(df, feature_name, nlags, lag_names_to_clip ):
        
        mean_feature_name = feature_name + '_mean'
        
        print('Adding lags for', mean_feature_name, '...')
    
        df_tmp = df[['date_block_num', 'shop_id', feature_name, mean_feature_name]]
    
        for i in range(nlags, 0, -1):
        
            lag_feature_name = mean_feature_name + '_lag-' + str(i)
            lag_names_to_clip.append(lag_feature_name)
        
            df_shifted = df_tmp.copy()
            df_shifted.columns = ['date_block_num', 'shop_id', feature_name, lag_feature_name]
            df_shifted['date_block_num'] += i
            df = pd.merge(df, df_shifted.drop_duplicates(), on=['date_block_num', 'shop_id', feature_name], how='left')
            df[lag_feature_name] = df[lag_feature_name].fillna(0).astype('float32')
        
        del df_tmp
        del df_shifted
        gc.collect()
        return df, lag_names_to_clip

### Adding lag features.........................

In [86]:
nlags=3

In [87]:
%%time

df, lag_names_to_clip =  add_item_lags(df,'item_cnt_month', nlags)
df = reduce_size(df)
df, lag_names_to_clip =  add_item_lags(df,'item_rate_month',nlags, lag_names_to_clip)
df = reduce_size(df)
df, lag_names_to_clip =  add_item_lags(df,'item_price', nlags,lag_names_to_clip)
df = reduce_size(df)


Adding item lags for item_cnt_month ...
Memory usage decreased to 444.86 Mb (10.2 % reduction)
Adding item lags for item_rate_month ...
Memory usage decreased to 495.23 Mb (9.2 % reduction)
Adding item lags for item_price ...
Memory usage decreased to 545.59 Mb (8.5 % reduction)
CPU times: user 1min 45s, sys: 937 ms, total: 1min 46s
Wall time: 1min 45s


In [88]:
%%time

df, lag_names_to_clip =  add_item_lags(df,'date_avg_item_cnt',nlags, lag_names_to_clip)
df, lag_names_to_clip =  add_item_lags(df,'date_item_avg_item_cnt',nlags, lag_names_to_clip)
df, lag_names_to_clip =  add_item_lags(df,'date_shop_avg_item_cnt',nlags, lag_names_to_clip)
df = reduce_size(df)
df, lag_names_to_clip =  add_item_lags(df,'date_shop_item_avg_item_cnt',nlags, lag_names_to_clip)
df = reduce_size(df)
df, lag_names_to_clip =  add_item_lags(df,'date_city_avg_item_cnt',nlags, lag_names_to_clip)
df, lag_names_to_clip =  add_item_lags(df,'date_item_city_avg_item_cnt',nlags, lag_names_to_clip)
df, lag_names_to_clip =  add_item_lags(df,'date_item_avg_item_price',nlags, lag_names_to_clip)

df = reduce_size(df)


Adding item lags for date_avg_item_cnt ...
Adding item lags for date_item_avg_item_cnt ...
Adding item lags for date_shop_avg_item_cnt ...
Memory usage decreased to 696.67 Mb (17.8 % reduction)
Adding item lags for date_shop_item_avg_item_cnt ...
Memory usage decreased to 747.04 Mb (6.3 % reduction)
Adding item lags for date_city_avg_item_cnt ...
Adding item lags for date_item_city_avg_item_cnt ...
Adding item lags for date_item_avg_item_price ...
Memory usage decreased to 898.12 Mb (14.4 % reduction)
CPU times: user 4min 21s, sys: 2.38 s, total: 4min 23s
Wall time: 4min 21s


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8801390 entries, 0 to 8801389
Data columns (total 54 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   date_block_num                     int8   
 1   shop_id                            int8   
 2   item_id                            int16  
 3   item_price                         float16
 4   item_cnt_month                     float16
 5   item_rate_month                    float16
 6   date_avg_item_cnt                  float16
 7   date_item_avg_item_cnt             float16
 8   date_shop_avg_item_cnt             float16
 9   date_shop_item_avg_item_cnt        float16
 10  city_enc                           int8   
 11  item_category_id                   int8   
 12  Category_id                        int8   
 13  date_city_avg_item_cnt             float16
 14  date_item_city_avg_item_cnt        float16
 15  item_avg_item_price                float16
 16  date_item_avg_item

### Creating Delta Lag Features...................

In [90]:
%%time
lags = [1,2,3]
for i in lags:
    df["delta_price_lag_" + str(i) ] = (df["date_item_avg_item_price_lag-" + str(i)]- df["item_avg_item_price"] )/ df["item_avg_item_price"]

def select_trends(row) :
    for i in lags:
        if row["delta_price_lag_" + str(i)]:
            return row["delta_price_lag_" + str(i)]
    return 0

df["delta_price_lag"] = df.apply(select_trends, axis = 1)
df["delta_price_lag"] = df.delta_price_lag.astype( np.float16 )
df["delta_price_lag"].fillna( 0 ,inplace = True)

CPU times: user 2min 14s, sys: 11.9 s, total: 2min 26s
Wall time: 2min 25s


## Dropping Features...

### Dropping features .................................

In [91]:
features_to_drop = ["item_avg_item_price", "date_item_avg_item_price"]

In [92]:
for i in lags:
    features_to_drop.append("date_item_avg_item_price_lag-" + str(i) )
    features_to_drop.append("delta_price_lag_" + str(i) )
df.drop(features_to_drop, axis = 1, inplace = True)

In [93]:
# Removing previously dropped lag items
remve = ['date_item_avg_item_price_lag-1','date_item_avg_item_price_lag-2','date_item_avg_item_price_lag-3']

for x in remve:
  lag_names_to_clip.remove(x)
lag_names_to_clip

['item_cnt_month_lag-3',
 'item_cnt_month_lag-2',
 'item_cnt_month_lag-1',
 'item_rate_month_lag-3',
 'item_rate_month_lag-2',
 'item_rate_month_lag-1',
 'item_price_lag-3',
 'item_price_lag-2',
 'item_price_lag-1',
 'date_avg_item_cnt_lag-3',
 'date_avg_item_cnt_lag-2',
 'date_avg_item_cnt_lag-1',
 'date_item_avg_item_cnt_lag-3',
 'date_item_avg_item_cnt_lag-2',
 'date_item_avg_item_cnt_lag-1',
 'date_shop_avg_item_cnt_lag-3',
 'date_shop_avg_item_cnt_lag-2',
 'date_shop_avg_item_cnt_lag-1',
 'date_shop_item_avg_item_cnt_lag-3',
 'date_shop_item_avg_item_cnt_lag-2',
 'date_shop_item_avg_item_cnt_lag-1',
 'date_city_avg_item_cnt_lag-3',
 'date_city_avg_item_cnt_lag-2',
 'date_city_avg_item_cnt_lag-1',
 'date_item_city_avg_item_cnt_lag-3',
 'date_item_city_avg_item_cnt_lag-2',
 'date_item_city_avg_item_cnt_lag-1']

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8801390 entries, 0 to 8801389
Data columns (total 50 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   date_block_num                     int8   
 1   shop_id                            int8   
 2   item_id                            int16  
 3   item_price                         float16
 4   item_cnt_month                     float16
 5   item_rate_month                    float16
 6   date_avg_item_cnt                  float16
 7   date_item_avg_item_cnt             float16
 8   date_shop_avg_item_cnt             float16
 9   date_shop_item_avg_item_cnt        float16
 10  city_enc                           int8   
 11  item_category_id                   int8   
 12  Category_id                        int8   
 13  date_city_avg_item_cnt             float16
 14  date_item_city_avg_item_cnt        float16
 15  first_sale_date                    int8   
 16  month             

## Miscellaneuos

### Sampling final dataframe

In [95]:
df.sample(20)

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,...,date_shop_item_avg_item_cnt_lag-3,date_shop_item_avg_item_cnt_lag-2,date_shop_item_avg_item_cnt_lag-1,date_city_avg_item_cnt_lag-3,date_city_avg_item_cnt_lag-2,date_city_avg_item_cnt_lag-1,date_item_city_avg_item_cnt_lag-3,date_item_city_avg_item_cnt_lag-2,date_item_city_avg_item_cnt_lag-1,delta_price_lag
6272289,23,10,12048,0.0,0.0,0.0,0.0,0.048767,0.158813,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
850006,3,10,5870,0.0,0.0,0.0,0.0,0.027771,0.098694,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2473967,8,58,12182,0.0,0.0,0.0,0.0,0.513672,0.363770,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6996736,26,19,1881,3466.0,3.0,3.0,3.0,5.074219,0.259521,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4647398,17,25,5416,0.0,0.0,0.0,0.0,0.024994,0.860840,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2502529,9,41,12972,199.0,3.0,3.0,3.0,0.297363,0.246460,3.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3984227,14,19,19110,0.0,0.0,0.0,0.0,0.025635,0.309814,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1549639,5,26,12824,0.0,0.0,0.0,0.0,0.027023,0.280273,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
700584,2,10,3619,0.0,0.0,0.0,0.0,0.583496,0.125610,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5641923,21,31,16025,3000.0,1.0,1.0,1.0,0.097534,1.199219,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Checking for any data leakage in Test Data Set....................

In [96]:
X_test_temp = df[df["date_block_num"]==34]
X_test_temp[item_mean_features].sum()
X_test_temp[shop_mean_features].sum()

city_enc    3151800
dtype: int64

In [97]:
lag_names_to_clip

['item_cnt_month_lag-3',
 'item_cnt_month_lag-2',
 'item_cnt_month_lag-1',
 'item_rate_month_lag-3',
 'item_rate_month_lag-2',
 'item_rate_month_lag-1',
 'item_price_lag-3',
 'item_price_lag-2',
 'item_price_lag-1',
 'date_avg_item_cnt_lag-3',
 'date_avg_item_cnt_lag-2',
 'date_avg_item_cnt_lag-1',
 'date_item_avg_item_cnt_lag-3',
 'date_item_avg_item_cnt_lag-2',
 'date_item_avg_item_cnt_lag-1',
 'date_shop_avg_item_cnt_lag-3',
 'date_shop_avg_item_cnt_lag-2',
 'date_shop_avg_item_cnt_lag-1',
 'date_shop_item_avg_item_cnt_lag-3',
 'date_shop_item_avg_item_cnt_lag-2',
 'date_shop_item_avg_item_cnt_lag-1',
 'date_city_avg_item_cnt_lag-3',
 'date_city_avg_item_cnt_lag-2',
 'date_city_avg_item_cnt_lag-1',
 'date_item_city_avg_item_cnt_lag-3',
 'date_item_city_avg_item_cnt_lag-2',
 'date_item_city_avg_item_cnt_lag-1']

### Checking unique numbers in every feature/column of dataframe

In [98]:
for x in lag_names_to_clip:
  print(f'Feature {x}: Unique numbers are: {df[x].nunique()}')

Feature item_cnt_month_lag-3: Unique numbers are: 21
Feature item_cnt_month_lag-2: Unique numbers are: 21
Feature item_cnt_month_lag-1: Unique numbers are: 21
Feature item_rate_month_lag-3: Unique numbers are: 32
Feature item_rate_month_lag-2: Unique numbers are: 32
Feature item_rate_month_lag-1: Unique numbers are: 32
Feature item_price_lag-3: Unique numbers are: 6642
Feature item_price_lag-2: Unique numbers are: 6782
Feature item_price_lag-1: Unique numbers are: 6885
Feature date_avg_item_cnt_lag-3: Unique numbers are: 21
Feature date_avg_item_cnt_lag-2: Unique numbers are: 21
Feature date_avg_item_cnt_lag-1: Unique numbers are: 21
Feature date_item_avg_item_cnt_lag-3: Unique numbers are: 1913
Feature date_item_avg_item_cnt_lag-2: Unique numbers are: 1928
Feature date_item_avg_item_cnt_lag-1: Unique numbers are: 2038
Feature date_shop_avg_item_cnt_lag-3: Unique numbers are: 1001
Feature date_shop_avg_item_cnt_lag-2: Unique numbers are: 1021
Feature date_shop_avg_item_cnt_lag-1: Uniqu

### Fill 'zero' vaues in item_price_lag 1 & 2 with values in item_price_lag 2 & 3

In [99]:
idx = df[(df['item_price_lag-1'] == 0) & (df['item_price_lag-2'] != 0)].index
df.loc[idx, 'item_price_lag-1'] = df.loc[idx, 'item_price_lag-2']
idx = df[(df['item_price_lag-2'] == 0) & (df['item_price_lag-3'] != 0)].index
df.loc[idx, 'item_price_lag-2'] = df.loc[idx, 'item_price_lag-3']
        

### Creating 'gradient' and 'lag_feature_mean' features from lag features

In [100]:
df['gradient_1'] = df['item_cnt_month_lag-1']/df['item_cnt_month_lag-2']
df['gradient_1'] = df['gradient_1'].replace([np.inf, -np.inf], np.nan).fillna(0.)

df['gradient_2'] = df['item_cnt_month_lag-2']/df['item_cnt_month_lag-3']
df['gradient_2'] = df['gradient_2'].replace([np.inf, -np.inf], np.nan).fillna(0.)

In [101]:
df['lag_feature_mean'] = df[['item_cnt_month_lag-1', 
                                    'item_cnt_month_lag-2', 
                                    'item_cnt_month_lag-3']].mean(skipna=True, axis=1)

In [102]:
df["delta_price_lag"] = df["delta_price_lag"].replace([np.inf, -np.inf], np.nan).fillna(0.)

In [103]:
df.drop('new_item',axis=1,inplace=True)


In [104]:
df

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,...,date_city_avg_item_cnt_lag-3,date_city_avg_item_cnt_lag-2,date_city_avg_item_cnt_lag-1,date_item_city_avg_item_cnt_lag-3,date_item_city_avg_item_cnt_lag-2,date_item_city_avg_item_cnt_lag-1,delta_price_lag,gradient_1,gradient_2,lag_feature_mean
0,0,59,22154,999.0,1.0,1.0,1.0,0.484863,0.267578,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0,59,2554,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0,59,2555,0.0,0.0,0.0,0.0,0.030304,0.267578,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0,59,2564,0.0,0.0,0.0,0.0,0.121216,0.267578,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0,59,2565,0.0,0.0,0.0,0.0,0.060608,0.267578,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801385,34,45,18454,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333252
8801386,34,45,16188,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8801387,34,45,15757,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8801388,34,45,19648,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [105]:
df[df['date_block_num'] == 34]

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_rate_month,date_avg_item_cnt,date_item_avg_item_cnt,date_shop_avg_item_cnt,date_shop_item_avg_item_cnt,...,date_city_avg_item_cnt_lag-3,date_city_avg_item_cnt_lag-2,date_city_avg_item_cnt_lag-1,date_item_city_avg_item_cnt_lag-3,date_item_city_avg_item_cnt_lag-2,date_item_city_avg_item_cnt_lag-1,delta_price_lag,gradient_1,gradient_2,lag_feature_mean
8587190,34,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.333252,1.333008
8587191,34,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8587192,34,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333252,3.000000,1.666992
8587193,34,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.333252
8587194,34,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801385,34,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.333252
8801386,34,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8801387,34,45,15757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8801388,34,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


### Creating new feature of 'noisy_shop'

In [106]:
noisy_shops = [25, 31, 42]
df['noisy_shop'] = df['shop_id'].isin(noisy_shops)

### Dropping features..............

In [107]:
# df.drop(['item_price','item_rate_month','item_cnt_mean','city_enc_mean','item_category_id_mean'],axis=1,inplace=True)

df.drop(['item_price','item_cnt_mean','city_enc_mean','item_category_id_mean','item_rate_month'],axis=1,inplace=True)


### Checking df info()

In [108]:
df = reduce_size(df)

Memory usage decreased to 1055.40 Mb (8.7 % reduction)


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8801390 entries, 0 to 8801389
Data columns (total 48 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   date_block_num                     int8   
 1   shop_id                            int8   
 2   item_id                            int16  
 3   item_cnt_month                     float16
 4   date_avg_item_cnt                  float16
 5   date_item_avg_item_cnt             float16
 6   date_shop_avg_item_cnt             float16
 7   date_shop_item_avg_item_cnt        float16
 8   city_enc                           int8   
 9   item_category_id                   int8   
 10  Category_id                        int8   
 11  date_city_avg_item_cnt             float16
 12  date_item_city_avg_item_cnt        float16
 13  first_sale_date                    int8   
 14  month                              int8   
 15  first_instance_of_shop             int8   
 16  item_cnt_month_lag

In [110]:
df.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'date_avg_item_cnt', 'date_item_avg_item_cnt', 'date_shop_avg_item_cnt',
       'date_shop_item_avg_item_cnt', 'city_enc', 'item_category_id',
       'Category_id', 'date_city_avg_item_cnt', 'date_item_city_avg_item_cnt',
       'first_sale_date', 'month', 'first_instance_of_shop',
       'item_cnt_month_lag-3', 'item_cnt_month_lag-2', 'item_cnt_month_lag-1',
       'item_rate_month_lag-3', 'item_rate_month_lag-2',
       'item_rate_month_lag-1', 'item_price_lag-3', 'item_price_lag-2',
       'item_price_lag-1', 'date_avg_item_cnt_lag-3',
       'date_avg_item_cnt_lag-2', 'date_avg_item_cnt_lag-1',
       'date_item_avg_item_cnt_lag-3', 'date_item_avg_item_cnt_lag-2',
       'date_item_avg_item_cnt_lag-1', 'date_shop_avg_item_cnt_lag-3',
       'date_shop_avg_item_cnt_lag-2', 'date_shop_avg_item_cnt_lag-1',
       'date_shop_item_avg_item_cnt_lag-3',
       'date_shop_item_avg_item_cnt_lag-2',
       'date_shop_ite

### Dropping features..............

In [111]:
df.drop(['date_avg_item_cnt', 'date_item_avg_item_cnt', 'date_shop_avg_item_cnt',
       'date_shop_item_avg_item_cnt','date_city_avg_item_cnt', 'date_item_city_avg_item_cnt'],axis=1,inplace=True)


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8801390 entries, 0 to 8801389
Data columns (total 42 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   date_block_num                     int8   
 1   shop_id                            int8   
 2   item_id                            int16  
 3   item_cnt_month                     float16
 4   city_enc                           int8   
 5   item_category_id                   int8   
 6   Category_id                        int8   
 7   first_sale_date                    int8   
 8   month                              int8   
 9   first_instance_of_shop             int8   
 10  item_cnt_month_lag-3               float16
 11  item_cnt_month_lag-2               float16
 12  item_cnt_month_lag-1               float16
 13  item_rate_month_lag-3              float16
 14  item_rate_month_lag-2              float16
 15  item_rate_month_lag-1              float16
 16  item_price_lag-3  

In [113]:
df.isna().sum()

date_block_num                       0
shop_id                              0
item_id                              0
item_cnt_month                       0
city_enc                             0
item_category_id                     0
Category_id                          0
first_sale_date                      0
month                                0
first_instance_of_shop               0
item_cnt_month_lag-3                 0
item_cnt_month_lag-2                 0
item_cnt_month_lag-1                 0
item_rate_month_lag-3                0
item_rate_month_lag-2                0
item_rate_month_lag-1                0
item_price_lag-3                     0
item_price_lag-2                     0
item_price_lag-1                     0
date_avg_item_cnt_lag-3              0
date_avg_item_cnt_lag-2              0
date_avg_item_cnt_lag-1              0
date_item_avg_item_cnt_lag-3         0
date_item_avg_item_cnt_lag-2         0
date_item_avg_item_cnt_lag-1         0
date_shop_avg_item_cnt_la

## Pickling the file

In [120]:
import pickle
df.to_pickle('/content/drive/MyDrive/Applied AI/Capstone/Predict Future Sales/Final_Files/pickled_df.pkl')

In [121]:
unpickled_df = pd.read_pickle('/content/drive/MyDrive/Applied AI/Capstone/Predict Future Sales/Final_Files/pickled_df.pkl')
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8801390 entries, 0 to 8801389
Data columns (total 42 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   date_block_num                     int8   
 1   shop_id                            int8   
 2   item_id                            int16  
 3   item_cnt_month                     float16
 4   city_enc                           int8   
 5   item_category_id                   int8   
 6   Category_id                        int8   
 7   first_sale_date                    int8   
 8   month                              int8   
 9   first_instance_of_shop             int8   
 10  item_cnt_month_lag-3               float16
 11  item_cnt_month_lag-2               float16
 12  item_cnt_month_lag-1               float16
 13  item_rate_month_lag-3              float16
 14  item_rate_month_lag-2              float16
 15  item_rate_month_lag-1              float16
 16  item_price_lag-3  

In [122]:
unpickled_df.head()

,date_block_num,shop_id,item_id,item_cnt_month,city_enc,item_category_id,Category_id,first_sale_date,month,first_instance_of_shop,...,date_city_avg_item_cnt_lag-2,date_city_avg_item_cnt_lag-1,date_item_city_avg_item_cnt_lag-3,date_item_city_avg_item_cnt_lag-2,date_item_city_avg_item_cnt_lag-1,delta_price_lag,gradient_1,gradient_2,lag_feature_mean,noisy_shop
0,0,59,22154,1.0,29,37,7,34,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1,0,59,2554,0.0,29,58,9,15,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,0,59,2555,0.0,29,56,9,34,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,0,59,2564,0.0,29,59,9,34,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,0,59,2565,0.0,29,56,9,34,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
